This example is taken from [TensorFlow Recommenders: Quickstart](https://www.tensorflow.org/recommenders/examples/quickstart) as is with one change: Adam optimizer is replaced with Adan optimizer. 

##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Recommenders: Quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/quickstart"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

### Import TFRS

First, install and import TFRS:

In [2]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [3]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2022-10-01 18:17:28.630616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-01 18:17:28.790996: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-01 18:17:28.791029: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-01 18:17:28.824790: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-01 18:17:29.668342: W tensorflow/stream_executor/platform/de

In [4]:
from tf_adan.adan import Adan

### Read the data

In [5]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

2022-10-01 18:17:31.922397: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|          | 0/4 [00:00<?, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]
Dl Size...:  25%|██▌       | 1/4 [00:01<00:05,  1.78s/ MiB]

Extraction completed...: 0 file [00:01, ? file/s]
Dl Size...:  50%|█████     | 2/4 [00:02<00:02,  1.03s/ MiB]

Extraction completed...: 0 file [00:02, ? file/s]
Dl Size...:  75%|███████▌  | 3/4 [00:02<00:00,  1.44 MiB/s]

Extraction completed...: 0 file [00:02, ? file/s]
Dl Size...: 100%|██████████| 4/4 [00:03<00:00,  1.48 MiB/s]

Dl Size...: 100%|██████████| 4/4 [00:03<00:00,  1.48 MiB/s]

Dl Size...: 100%|██████████| 4/4 [00:03<00:00,  1.48 MiB/s]

Extraction completed...:   0%|          | 0/1 [00:03<?, ? file/s]

Dl Size...: 100%|██████████| 4/4 [00:03<00:00,  1.48 MiB/s]

Generating train examples...:  15%|█▍        | 14733/100000 [00:06<00:39, 2179.23 ex

Generating train examples...:  34%|███▍      | 33847/100000 [00:15<00:31, 2100.39 examples/s]


Generating train examples...:  53%|█████▎    | 52662/100000 [00:24<00:20, 2358.28 examples/s]


Generating train examples...:  74%|███████▎  | 73705/100000 [00:33<00:10, 2440.38 examples/s]


...

Generating train examples...: 100%|█████████▉| 99910/100000 [00:43<00:00, 2258.75 examples/s]
                                                                                             
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:   0%|          | 0/100000 [00:00<?, ? examples/s]
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:   0%|          | 438/100000 [00:00<00:22, 4369.87 examples/s]
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:   3%|▎         | 2787/100000 [00:00<00:06, 15593.61 examples/s]
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:   5%|▌         | 5006/100000 [00:00<00:05, 18591.64 examples/s]
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:   7%|▋         | 6866/100000 [00

Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:  80%|████████  | 80124/100000 [00:03<00:00, 20345.20 examples/s]
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:  82%|████████▏ | 82381/100000 [00:03<00:00, 20977.21 examples/s]
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:  85%|████████▍ | 84717/100000 [00:03<00:00, 21644.87 examples/s]
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:  87%|████████▋ | 86972/100000 [00:04<00:00, 21885.42 examples/s]
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:  89%|████████▉ | 89217/100000 [00:04<00:00, 22049.33 examples/s]
Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete45A7EM/movielens-train.tfrecord*...:  91%|█████████▏| 91432/1000

Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 57.54 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|          | 0/4924029 [00:00<?, ? MiB/s]

Dl Size...: 100%|██████████| 4924029/4924029 [00:00<00:00, 146610500.04 MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]
Generating train examples...:  79%|███████▊  | 1323/1682 [00:00<00:00, 2660.48 examples/s]
                                                                                          
Shuffling ~/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteW8V83X/movielens-train.tfrecord*...:   0%|          | 0/1682 [00:00<?, ? examples/s]


Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [6]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Define a model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [7]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

Define the two models and the retrieval task.

In [8]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)


### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [9]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=Adan())

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3
25/25 [==============================] - 14s 472ms/step - factorized_top_k/top_1_categorical_accuracy: 2.6000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0023 - factorized_top_k/top_10_categorical_accuracy: 0.0052 - factorized_top_k/top_50_categorical_accuracy: 0.0286 - factorized_top_k/top_100_categorical_accuracy: 0.0572 - loss: 32419.4401 - regularization_loss: 0.0000e+00 - total_loss: 32419.4401
Epoch 2/3
25/25 [==============================] - 18s 705ms/step - factorized_top_k/top_1_categorical_accuracy: 2.1000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0041 - factorized_top_k/top_10_categorical_accuracy: 0.0085 - factorized_top_k/top_50_categorical_accuracy: 0.0418 - factorized_top_k/top_100_categorical_accuracy: 0.0790 - loss: 32414.9437 - regularization_loss: 0.0000e+00 - total_loss: 32414.9437
Epoch 3/3
25/25 [==============================] - 16s 628ms/step - factorized_top_k/top_1_categorical_accuracy: 4.1000e-04 - factorized_top_k/top_5_categor